In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [4]:
books.head()

isbn13      isbn10                title  \
0  9780002005883  0002005883               Gilead   
1  9780002261982  0002261987         Spider's Web   
2  9780006178736  0006178731       Rage of angels   
3  9780006280897  0006280897       The Four Loves   
4  9780006280934  0006280935  The Problem of Pain   

                           authors                     categories  \
0               Marilynne Robinson                        Fiction   
1  Charles Osborne;Agatha Christie  Detective and mystery stories   
2                   Sidney Sheldon                        Fiction   
3              Clive Staples Lewis                 Christian life   
4              Clive Staples Lewis                 Christian life   

                                           thumbnail  \
0  http://books.google.com/books/content?id=KQZCP...   
1  http://books.google.com/books/content?id=gA5GP...   
2  http://books.google.com/books/content?id=FKo2T...   
3  http://books.google.com/books/content?id=XhQ5X...   
4  http://books.google.com/books/content?id=Kk-uV...   

                                         description  published_year  \
0  A NOVEL THAT READERS and critics have been eag...          2004.0   
1  A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2  A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3  Lewis' work on the nature of love divides love...          2002.0   
4  "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   

   average_rating  num_pages  ratings_count     title_and_subtitle  \
0            3.85      247.0          361.0                 Gilead   
1            3.83      241.0         5164.0  Spider's Web: A Novel   
2            3.93      512.0        29532.0         Rage of angels   
3            4.15      170.0        33684.0         The Four Loves   
4            4.09      176.0        37569.0    The Problem of Pain   

                                  tagged_description  
0  9780002005883 A NOVEL THAT READERS and critics...  
1  9780002261982 A new 'Christie for Christmas' -...  
2  9780006178736 A memorable, mesmerizing heroine...  
3  9780006280897 Lewis' work on the nature of lov...  
4  9780006280934 "In The Problem of Pain, C.S. Le...

In [10]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [11]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n",
                                   index=False,
                                   header=False)

In [5]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1182, which is longer than the specified 1
Created a chunk of size 1214, which is longer than the specified 1
Created a chunk of size 373, which is longer than the specified 1
Created a chunk of size 309, which is longer than the specified 1
Created a chunk of size 483, which is longer than the specified 1
Created a chunk of size 482, which is longer than the specified 1
Created a chunk of size 960, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 847, which is longer than the specified 1
Created a chunk of size 296, which is longer than the specified 1
Created a chunk of size 197, which is longer than the specified 1
Created a chunk of size 881, which is longer than the specified 1
Created a chunk of size 1088, which is longer than the specified 1
Created a chunk of size 1189, which is longer than the specified 1
Created a chunk of size 304, which is longer than the specified 1
Create

In [7]:
documents[10]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780006483892 Available in the U.S. for the first time, this is the second volume in the exceptional Legends of the Riftwar series from ""New York Times""-bestselling authors Feist and Rosenberg."')

In [ ]:
## run it just once
import os

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
db_books = Chroma.from_documents(
    documents,
    embedding=embeddings,
    persist_directory="./chroma_db") ## save the database so we do not need to call the API every time

In [ ]:
## load the database
db_books = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings
)

In [25]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10) # k here means return the top 10 most similar results
docs

[Document(id='f5f7e9ac-d4d3-4d44-86af-0f139b33c65d', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='5f7aa143-d7df-4cb6-a9c0-cae8e536a3a2', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='2d8e7429-5183-40df-9f9b-251c700cee48', metadata={'source': '

In [17]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

isbn13      isbn10                                title  \
3747  9780786808069  0786808063  Baby Einstein: Neighborhood Animals   

                                authors        categories  \
3747  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3747  http://books.google.com/books/content?id=X9a4P...   

                                            description  published_year  \
3747  Children will discover the exciting world of t...          2001.0   

      average_rating  num_pages  ratings_count  \
3747            3.89       16.0          180.0   

                       title_and_subtitle  \
3747  Baby Einstein: Neighborhood Animals   

                                     tagged_description  
3747  9780786808069 Children will discover the excit...

In [18]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)


In [20]:
retrieve_semantic_recommendations(query)

isbn13      isbn10                                title  \
3214  9780689861130  0689861133                  Moo, Baa, la la La!   
3747  9780786808069  0786808063  Baby Einstein: Neighborhood Animals   
3748  9780786808373  0786808373                 Baby Einstein: Birds   
3749  9780786808380  0786808381                Baby Einstein: Babies   
3750  9780786808397  078680839X                  Baby Einstein: Dogs   

                                authors        categories  \
3214                     Sandra Boynton     Animal sounds   
3747  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   
3748                 Julie Aigner-Clark  Juvenile Fiction   
3749                 Julie Aigner-Clark  Juvenile Fiction   
3750                 Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3214  http://books.google.com/books/content?id=Gz40A...   
3747  http://books.google.com/books/content?id=X9a4P...   
3748  http://books.google.com/books/content?id=0jxHP...   
3749  http://books.google.com/books/content?id=jv4NA...   
3750  http://books.google.com/books/content?id=qut8t...   

                                            description  published_year  \
3214  Children will love joining in and imitating th...          2004.0   
3747  Children will discover the exciting world of t...          2001.0   
3748  Introducing your baby to birds, cats, dogs, an...          2002.0   
3749  Introduce your babies to birds, cats, dogs, an...          2002.0   
3750  Introduce your baby to birds, cats, dogs, and ...          2002.0   

      average_rating  num_pages  ratings_count  \
3214            4.20       14.0        28261.0   
3747            3.89       16.0          180.0   
3748            3.78       20.0            9.0   
3749            4.03       20.0           29.0   
3750            3.81       20.0           26.0   

                       title_and_subtitle  \
3214                  Moo, Baa, la la La!   
3747  Baby Einstein: Neighborhood Animals   
3748                 Baby Einstein: Birds   
3749                Baby Einstein: Babies   
3750                  Baby Einstein: Dogs   

                                     tagged_description  
3214  9780689861130 Children will love joining in an...  
3747  9780786808069 Children will discover the excit...  
3748  9780786808373 Introducing your baby to birds, ...  
3749  9780786808380 Introduce your babies to birds, ...  
3750  9780786808397 Introduce your baby to birds, ca...